# MEWpy example

Increasing the production of succinate in E.coli by modifying genes expression.

We start by loading the Genome-scale Metabolic Model using REFRAMED or COBRApy.

In [8]:
from cobra.io.sbml import read_sbml_model

model = read_sbml_model('models/ec/iJO1366.xml', flavor='cobra')


We import the necesseray modules.

In [9]:
from mewpy.optimization.evaluation import BPCY,WYIELD
from mewpy.problems import GOUProblem
from mewpy.simulation import get_simulator

We may have a quick look into the reactions included in the model

In [10]:
sim = get_simulator(model)
sim.find(['BIOMASS','EX_o2_','EX_glc_','EX_succ_'])

,name,lb,ub,stoichiometry,gpr,annotations
id,,,,,,
EX_glc__D_e,D-Glucose exchange,-10.0,1000.0,{'glc__D_e': -1.0},,"{'sbo': 'SBO:0000627', 'bigg.reaction': 'EX_gl..."
BIOMASS_Ec_iJO1366_WT_53p95M,E. coli biomass objective function (iJO1366) -...,0.0,1000.0,"{'10fthf_c': -0.000223, '2dmmql8_c': -0.000223...",,"{'sbo': 'SBO:0000629', 'bigg.reaction': 'BIOMA..."
BIOMASS_Ec_iJO1366_core_53p95M,E. coli biomass objective function (iJO1366) -...,0.0,1000.0,"{'10fthf_c': -0.000223, '2fe2s_c': -2.6e-05, '...",,"{'sbo': 'SBO:0000629', 'bigg.reaction': 'BIOMA..."
EX_o2_e,O2 exchange,-1000.0,1000.0,{'o2_e': -1.0},,"{'sbo': 'SBO:0000627', 'bigg.reaction': 'EX_o2..."
EX_succ_e,Succinate exchange,0.0,1000.0,{'succ_e': -1.0},,"{'sbo': 'SBO:0000627', 'bigg.reaction': 'EX_su..."


We can now define the optimization problem by defining the medium and objective functions, BPCY and WYIELD.

In [11]:
BIOMASS_ID = 'BIOMASS_Ec_iJO1366_core_53p95M'
O2 = 'EX_o2_e'
GLC = 'EX_glc__D_e'
PRODUCT_ID = 'EX_succ_e'
sim.objective = BIOMASS_ID

evaluator_1 = BPCY(BIOMASS_ID, PRODUCT_ID, method = 'lMOMA')
evaluator_2 = WYIELD(BIOMASS_ID,PRODUCT_ID)

medium = {GLC:(-10,1000),O2:(-20,1000)}

We instantiate the problem, a gene over/under expression problem.

In [12]:
problem = GOUProblem(model,
                     [evaluator_1,evaluator_2],
                     envcond=medium,
                     candidate_min_size=2,
                     candidate_max_size=6
                     )

An EA optimizer is instantiated for the defined optimization problem and ran. The `max_generations` value depends on the difficulty and size of the solution space. For illustrative purposes, we only will consider 10 iterations.

In [13]:
from mewpy.optimization import EA
ea = EA(problem, max_generations= 10, mp = True)
final_pop = ea.run()

Running NSGAII
Eval(s)|     Worst      Best    Median   Average   Std Dev|     Worst      Best    Median   Average   Std Dev|
    100|  -0.000000  0.000000  0.000000  0.000000  0.000000|  0.000000  4.495700  0.051676  0.448541  1.201753|
    200|  -0.000000  0.000000  0.000000  0.000000  0.000000|  0.000000  4.495700  0.051676  1.010836  1.691651|
    300|  -0.000000  0.000000  0.000000  -0.000000  0.000000|  0.000000  4.495700  3.862452  2.753325  1.858796|


SystemExit: 0

/home/vmsapereira/anaconda3/envs/py37/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3465: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [ ]:
ea.dataframe()

In [ ]:
final_pop[0].values

In [ ]:
res = problem.simulate(solution=final_pop[0].values,method='pFBA')
res.find([PRODUCT_ID,BIOMASS_ID])


In [ ]:
problem.FVA(PRODUCT_ID,solution=final_pop[0].values)